In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

2023-05-28 12:06:07.648344: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
class Sampling(layers.Layer):
    """Uses (z_mean, z_log_var) to sample z, the vector encoding a digit."""

    def call(self, inputs):
        z_mean, z_log_var = inputs
        batch = tf.shape(z_mean)[0]
        dim = tf.shape(z_mean)[1]
        epsilon = tf.keras.backend.random_normal(shape=(batch, dim))
        return z_mean + tf.exp(0.5 * z_log_var) * epsilon


In [3]:
latent_dim = 2

encoder_inputs = keras.Input(shape=(28, 28, 1))
x = layers.Conv2D(32, 3, activation="relu", strides=2, padding="same")(encoder_inputs)
x = layers.Conv2D(64, 3, activation="relu", strides=2, padding="same")(x)
x = layers.Flatten()(x)
x = layers.Dense(16, activation="relu")(x)
z_mean = layers.Dense(latent_dim, name="z_mean")(x)
z_log_var = layers.Dense(latent_dim, name="z_log_var")(x)
z = Sampling()([z_mean, z_log_var])
encoder = keras.Model(encoder_inputs, [z_mean, z_log_var, z], name="encoder")
encoder.summary()


2023-05-27 10:33:57.707973: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.


Model: "encoder"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 28, 28, 1)]  0           []                               
                                                                                                  
 conv2d (Conv2D)                (None, 14, 14, 32)   320         ['input_1[0][0]']                
                                                                                                  
 conv2d_1 (Conv2D)              (None, 7, 7, 64)     18496       ['conv2d[0][0]']                 
                                                                                                  
 flatten (Flatten)              (None, 3136)         0           ['conv2d_1[0][0]']               
                                                                                            

In [4]:
latent_inputs = keras.Input(shape=(latent_dim,))
x = layers.Dense(7 * 7 * 64, activation="relu")(latent_inputs)
x = layers.Reshape((7, 7, 64))(x)
x = layers.Conv2DTranspose(64, 3, activation="relu", strides=2, padding="same")(x)
x = layers.Conv2DTranspose(32, 3, activation="relu", strides=2, padding="same")(x)
decoder_outputs = layers.Conv2DTranspose(1, 3, activation="sigmoid", padding="same")(x)
decoder = keras.Model(latent_inputs, decoder_outputs, name="decoder")
decoder.summary()

Model: "decoder"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 2)]               0         
                                                                 
 dense_1 (Dense)             (None, 3136)              9408      
                                                                 
 reshape (Reshape)           (None, 7, 7, 64)          0         
                                                                 
 conv2d_transpose (Conv2DTra  (None, 14, 14, 64)       36928     
 nspose)                                                         
                                                                 
 conv2d_transpose_1 (Conv2DT  (None, 28, 28, 32)       18464     
 ranspose)                                                       
                                                                 
 conv2d_transpose_2 (Conv2DT  (None, 28, 28, 1)        289 

In [5]:
class VAE(keras.Model):
    def __init__(self, encoder, decoder, **kwargs):
        super().__init__(**kwargs)
        self.encoder = encoder
        self.decoder = decoder
        self.total_loss_tracker = keras.metrics.Mean(name="total_loss")
        self.reconstruction_loss_tracker = keras.metrics.Mean(
            name="reconstruction_loss"
        )
        self.kl_loss_tracker = keras.metrics.Mean(name="kl_loss")

    @property
    def metrics(self):
        return [
            self.total_loss_tracker,
            self.reconstruction_loss_tracker,
            self.kl_loss_tracker,
        ]

    def train_step(self, data):
        with tf.GradientTape() as tape:
            z_mean, z_log_var, z = self.encoder(data)
            reconstruction = self.decoder(z)
            reconstruction_loss = tf.reduce_mean(
                tf.reduce_sum(
                    keras.losses.binary_crossentropy(data, reconstruction), axis=(1, 2)
                )
            )
            kl_loss = -0.5 * (1 + z_log_var - tf.square(z_mean) - tf.exp(z_log_var))
            kl_loss = tf.reduce_mean(tf.reduce_sum(kl_loss, axis=1))
            total_loss = reconstruction_loss + kl_loss
        grads = tape.gradient(total_loss, self.trainable_weights)
        self.optimizer.apply_gradients(zip(grads, self.trainable_weights))
        self.total_loss_tracker.update_state(total_loss)
        self.reconstruction_loss_tracker.update_state(reconstruction_loss)
        self.kl_loss_tracker.update_state(kl_loss)
        return {
            "loss": self.total_loss_tracker.result(),
            "reconstruction_loss": self.reconstruction_loss_tracker.result(),
            "kl_loss": self.kl_loss_tracker.result(),
        }


In [6]:
(x_train, _), (x_test, _) = keras.datasets.mnist.load_data()
mnist_digits = np.concatenate([x_train, x_test], axis=0)
mnist_digits = np.expand_dims(mnist_digits, -1).astype("float32") / 255

vae = VAE(encoder, decoder)
vae.compile(optimizer=keras.optimizers.Adam())
vae.fit(mnist_digits, epochs=2, batch_size=128)


11490434/11490434 [==============================] - 2s 0us/step
Epoch 1/2
547/547 [==============================] - 66s 115ms/step - loss: 248.3755 - reconstruction_loss: 205.1570 - kl_loss: 3.4305
Epoch 2/2
547/547 [==============================] - 69s 126ms/step - loss: 178.6454 - reconstruction_loss: 167.3625 - kl_loss: 5.1407


In [3]:
import csv
import numpy as np

# Get the real data from https://www.kaggle.com/mlg-ulb/creditcardfraud/
fname = "/home/ali/vae/archive/creditcard.csv"

all_features = []
all_targets = []
with open(fname) as f:
    for i, line in enumerate(f):
        if i == 0:
            print("HEADER:", line.strip())
            continue  # Skip header
        fields = line.strip().split(",")
        all_features.append([float(v.replace('"', "")) for v in fields[:-1]])
        all_targets.append([int(fields[-1].replace('"', ""))])
        if i == 1:
            print("EXAMPLE FEATURES:", all_features[-1])

features = np.array(all_features, dtype="float32")
targets = np.array(all_targets, dtype="uint8")
print("features.shape:", features.shape)
print("targets.shape:", targets.shape)


HEADER: "Time","V1","V2","V3","V4","V5","V6","V7","V8","V9","V10","V11","V12","V13","V14","V15","V16","V17","V18","V19","V20","V21","V22","V23","V24","V25","V26","V27","V28","Amount","Class"
EXAMPLE FEATURES: [0.0, -1.3598071336738, -0.0727811733098497, 2.53634673796914, 1.37815522427443, -0.338320769942518, 0.462387777762292, 0.239598554061257, 0.0986979012610507, 0.363786969611213, 0.0907941719789316, -0.551599533260813, -0.617800855762348, -0.991389847235408, -0.311169353699879, 1.46817697209427, -0.470400525259478, 0.207971241929242, 0.0257905801985591, 0.403992960255733, 0.251412098239705, -0.018306777944153, 0.277837575558899, -0.110473910188767, 0.0669280749146731, 0.128539358273528, -0.189114843888824, 0.133558376740387, -0.0210530534538215, 149.62]
features.shape: (284807, 30)
targets.shape: (284807, 1)


In [4]:
num_val_samples = int(len(features) * 0.2)
train_features = features[:-num_val_samples]
train_targets = targets[:-num_val_samples]
val_features = features[-num_val_samples:]
val_targets = targets[-num_val_samples:]

print("Number of training samples:", len(train_features))
print("Number of validation samples:", len(val_features))


Number of training samples: 227846
Number of validation samples: 56961


In [5]:
counts = np.bincount(train_targets[:, 0])
print(
    "Number of positive samples in training data: {} ({:.2f}% of total)".format(
        counts[1], 100 * float(counts[1]) / len(train_targets)
    )
)

weight_for_0 = 1.0 / counts[0]
weight_for_1 = 1.0 / counts[1]

Number of positive samples in training data: 417 (0.18% of total)


In [6]:
mean = np.mean(train_features, axis=0)
train_features -= mean
val_features -= mean
std = np.std(train_features, axis=0)
train_features /= std
val_features /= std

In [7]:
class Sampling(layers.Layer):
    """Uses (z_mean, z_log_var) to sample z, the vector encoding a digit."""

    def call(self, inputs):
        z_mean, z_log_var = inputs
        batch = tf.shape(z_mean)[0]
        dim = tf.shape(z_mean)[1]
        epsilon = tf.keras.backend.random_normal(shape=(batch, dim))
        return z_mean + tf.exp(0.5 * z_log_var) * epsilon


latent_dim = 2

encoder_inputs = keras.Input(shape=(30,))
# x = layers.Conv2D(32, 3, activation="relu", strides=2, padding="same")(encoder_inputs)
# x = layers.Conv2D(64, 3, activation="relu", strides=2, padding="same")(x)
# x = layers.Flatten()(x)
x = layers.Dense(30, activation="tanh")(encoder_inputs)
x = layers.Dense(20, activation="tanh")(x)
x = layers.Dense(18, activation="tanh")(x)
x = layers.Dense(16, activation="tanh")(x)
z_mean = layers.Dense(latent_dim, name="z_mean")(x)
z_log_var = layers.Dense(latent_dim, name="z_log_var")(x)
z = Sampling()([z_mean, z_log_var])
encoder = keras.Model(encoder_inputs, [z_mean, z_log_var, z], name="encoder")

latent_inputs = keras.Input(shape=(latent_dim,))
# x = layers.Dense(7 * 7 * 64, activation="relu")(latent_inputs)
# x = layers.Reshape((7, 7, 64))(x)
# x = layers.Conv2DTranspose(64, 3, activation="relu", strides=2, padding="same")(x)
# x = layers.Conv2DTranspose(32, 3, activation="relu", strides=2, padding="same")(x)
# decoder_outputs = layers.Conv2DTranspose(1, 3, activation="sigmoid", padding="same")(x)
x = layers.Dense(16, activation="tanh")(latent_inputs)
x = layers.Dense(18, activation="tanh")(x)
x = layers.Dense(20, activation="tanh")(x)
decoder_outputs = layers.Dense(30, activation="tanh")(x)
decoder = keras.Model(latent_inputs, decoder_outputs, name="decoder")



2023-05-28 12:07:11.701677: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.


In [8]:
class VAE(keras.Model):
    def __init__(self, encoder, decoder, **kwargs):
        super().__init__(**kwargs)
        self.encoder = encoder
        self.decoder = decoder
        self.total_loss_tracker = keras.metrics.Mean(name="total_loss")
        self.reconstruction_loss_tracker = keras.metrics.Mean(
            name="reconstruction_loss"
        )
        self.kl_loss_tracker = keras.metrics.Mean(name="kl_loss")

    @property
    def metrics(self):
        return [
            self.total_loss_tracker,
            self.reconstruction_loss_tracker,
            self.kl_loss_tracker,
        ]

    def train_step(self, data):
        with tf.GradientTape() as tape:
            z_mean, z_log_var, z = self.encoder(data)
            reconstruction = self.decoder(z)
#             reconstruction_loss = tf.reduce_mean(
#                 tf.reduce_sum(
#                     keras.losses.binary_crossentropy(data, reconstruction), axis=(1, 2)
#                 )
#             )
            reconstruction_loss = tf.keras.losses.MeanSquaredError()(data,reconstruction)
            kl_loss = -0.5 * (1 + z_log_var - tf.square(z_mean) - tf.exp(z_log_var))
            kl_loss = tf.reduce_mean(tf.reduce_sum(kl_loss, axis=1))
            total_loss = reconstruction_loss + kl_loss
        grads = tape.gradient(total_loss, self.trainable_weights)
        self.optimizer.apply_gradients(zip(grads, self.trainable_weights))
        self.total_loss_tracker.update_state(total_loss)
        self.reconstruction_loss_tracker.update_state(reconstruction_loss)
        self.kl_loss_tracker.update_state(kl_loss)
        return {
            "loss": self.total_loss_tracker.result(),
            "reconstruction_loss": self.reconstruction_loss_tracker.result(),
            "kl_loss": self.kl_loss_tracker.result(),
        }


In [9]:

creditdata = np.concatenate([train_features, val_features], axis=0)
creditdata = np.expand_dims(creditdata, -1).astype("float32")

vae = VAE(encoder, decoder)
vae.compile(optimizer=keras.optimizers.Adam())
vae.fit(creditdata, epochs=10, batch_size=128)

Epoch 1/10
2226/2226 [==============================] - 12s 4ms/step - loss: 1.0353 - reconstruction_loss: 1.0053 - kl_loss: 0.0033
Epoch 2/10
2226/2226 [==============================] - 10s 4ms/step - loss: 0.9919 - reconstruction_loss: 1.0045 - kl_loss: 2.7358e-05
Epoch 3/10
2226/2226 [==============================] - 9s 4ms/step - loss: 0.9895 - reconstruction_loss: 1.0045 - kl_loss: 7.4854e-06
Epoch 4/10
2226/2226 [==============================] - 10s 5ms/step - loss: 0.9985 - reconstruction_loss: 1.0046 - kl_loss: 2.5930e-06
Epoch 5/10
2226/2226 [==============================] - 10s 4ms/step - loss: 1.0088 - reconstruction_loss: 1.0050 - kl_loss: 1.3118e-06
Epoch 6/10
2226/2226 [==============================] - 10s 4ms/step - loss: 1.0088 - reconstruction_loss: 1.0045 - kl_loss: 1.2981e-06
Epoch 7/10
2226/2226 [==============================] - 10s 4ms/step - loss: 0.9918 - reconstruction_loss: 1.0045 - kl_loss: 1.0868e-06
Epoch 8/10
2226/2226 [==============================]

In [24]:
val_features.shape

(56961, 30)

In [67]:
test = train_features[1,:]

test = test.reshape([1,30])

In [77]:
z_mean, z_log_var, z = vae.encoder(test)
reconstruction = vae.decoder(z)

reconstruction_loss = tf.keras.losses.MeanSquaredError()(test,reconstruction)

In [78]:
reconstruction_loss.numpy()

0.41741496

In [70]:
reconstruction.numpy()

array([[ 0.41229054,  0.02760492,  0.00990844, -0.13348393, -0.03093056,
         0.04025487, -0.02018991,  0.01525421, -0.0075708 ,  0.0047708 ,
         0.01919036, -0.07557505,  0.04402744, -0.0281549 , -0.02722586,
        -0.04202577,  0.0039289 , -0.01101142,  0.02211242,  0.00563244,
        -0.01162461,  0.01399416,  0.0240715 ,  0.01843469, -0.0130402 ,
        -0.07930604, -0.01005057, -0.0105467 ,  0.00252189, -0.00742954]],
      dtype=float32)

In [71]:
test

array([[-2.0008564 ,  0.64784056,  0.16869006, -0.01075485,  0.28652602,
         0.09012804, -0.08543706, -0.04230526,  0.0670673 , -0.2293523 ,
        -0.15108545,  1.4866744 ,  1.0515386 ,  0.4633228 , -0.18291788,
         0.62796277,  0.5300628 , -0.14867269, -0.1904353 , -0.16889915,
        -0.10145202, -0.29439253, -0.8689419 ,  0.17637353, -0.56267697,
         0.25427756,  0.25284111, -0.0230066 ,  0.03819847, -0.3517778 ]],
      dtype=float32)

In [14]:
rec_loss = []

for i in range(val_features.shape[0]):
    
    if val_targets[i] == 1:
        
        test = val_features[i,:]

        test = test.reshape([1,30])

        z_mean, z_log_var, z = vae.encoder(test)
        reconstruction = vae.decoder(z)

        reconstruction_loss = tf.keras.losses.MeanSquaredError()(test,reconstruction)
        
        rec_loss.append(reconstruction_loss.numpy())


        print("Rec Loss = ", reconstruction_loss.numpy(), " -------- Label = ", val_targets[i], " Case No. ", i)

rec_loss = np.array(rec_loss)
print("mean = ", np.mean(rec_loss))
print("std = ", np.std(rec_loss))

Rec Loss =  7.518621  -------- Label =  [1]  Case No.  1866
Rec Loss =  3.3214269  -------- Label =  [1]  Case No.  1884
Rec Loss =  10.046226  -------- Label =  [1]  Case No.  2230
Rec Loss =  6.4760523  -------- Label =  [1]  Case No.  2630
Rec Loss =  5.1012144  -------- Label =  [1]  Case No.  4132
Rec Loss =  5.3102617  -------- Label =  [1]  Case No.  5412
Rec Loss =  12.018083  -------- Label =  [1]  Case No.  6728
Rec Loss =  5.242398  -------- Label =  [1]  Case No.  6786
Rec Loss =  5.2372904  -------- Label =  [1]  Case No.  6787
Rec Loss =  6.823598  -------- Label =  [1]  Case No.  6859
Rec Loss =  8.673994  -------- Label =  [1]  Case No.  7770
Rec Loss =  8.768185  -------- Label =  [1]  Case No.  7788
Rec Loss =  8.026488  -------- Label =  [1]  Case No.  7798
Rec Loss =  7.537183  -------- Label =  [1]  Case No.  9261
Rec Loss =  4.942793  -------- Label =  [1]  Case No.  9580
Rec Loss =  15.527534  -------- Label =  [1]  Case No.  10376
Rec Loss =  4.081621  -------- 

In [15]:
val_features[6728,:]

array([  1.7462541 ,  -0.50769436,   2.6397295 ,  -4.164056  ,
         3.7277195 ,   1.124849  ,  -1.683369  ,  -1.2245529 ,
         0.51499057,  -3.5956087 ,  -5.744471  ,   3.3477724 ,
        -7.4962306 ,  -1.657718  , -12.934678  ,  -0.2816221 ,
        -1.6018841 ,  -3.7281365 ,   0.09827992,  -2.1891088 ,
         0.669623  ,   0.54346186,  -1.3017883 ,  -0.5808547 ,
        -0.36568624,   0.4449785 ,   0.09471209,   2.115266  ,
         1.5862252 ,  -0.36251542], dtype=float32)